In [29]:
from openpyxl import load_workbook
import pandas as pd

# Load the Excel workbook
book = load_workbook('data/Xi 2023-24 02_September.xlsx', data_only=False)

# Get the required sheet
sheet = book.active



In [48]:
import re
pd.set_option('display.max_rows', 1000)


# Create a data frame from the rows in the sheet
data = [row for row in sheet.values]
col_names = data[5]
data_rows = data[6:]

df = pd.DataFrame(data_rows, columns=col_names)
df = df.drop(columns=['foot'])
df = df.drop(df.index[0])


date_pattern = re.compile(r'^\d{4}-\d{2}-\d{2}.*') # \d stands for any digit (0-9), .* stands for any characters
first_invalid_date_index = df['Date'].apply(lambda x: pd.isnull(x) or not date_pattern.match(str(x))).idxmax()
print(first_invalid_date_index)

df = df.iloc[:first_invalid_date_index - 1]
df


166


,Date,None,Category,rcpt,Description,None,data,electricity,gas,vonage,...,social,large,rent,None,None,None,None,None,None,None
1,2023-07-03 00:00:00,-65.28,sum food,None,STAR MARKET 0602 06/29 PURCHASE BOSTON MA DEBI...,None,"=IF(H$6=$D8,$B8,0)","=IF(I$6=$D8,$B8,0)","=IF(J$6=$D8,$B8,0)","=IF(K$6=$D8,$B8,0)",...,"=IF(AA$6=$D8,$B8,0)","=IF(AB$6=$D8,$B8,0)","=IF(AC$6=$D8,$B8,0)",None,None,None,None,None,None,None
2,2023-07-03 00:00:00,-565.64,sum food,None,BJS.COM #5490 07/01 PURCHASE 800-257-2582 MA D...,None,"=IF(H$6=$D9,$B9,0)","=IF(I$6=$D9,$B9,0)","=IF(J$6=$D9,$B9,0)","=IF(K$6=$D9,$B9,0)",...,"=IF(AA$6=$D9,$B9,0)","=IF(AB$6=$D9,$B9,0)","=IF(AC$6=$D9,$B9,0)",None,None,None,None,None,None,None
3,2023-07-03 00:00:00,-82.69,sum food,None,GROCERY WEEE! 07/01 PURCHASE HTTPSWWW.SAYW CA ...,None,"=IF(H$6=$D10,$B10,0)","=IF(I$6=$D10,$B10,0)","=IF(J$6=$D10,$B10,0)","=IF(K$6=$D10,$B10,0)",...,"=IF(AA$6=$D10,$B10,0)","=IF(AB$6=$D10,$B10,0)","=IF(AC$6=$D10,$B10,0)",None,None,None,None,None,None,None
4,2023-07-03 00:00:00,-276.39,sum food,None,BJS.COM #5490 07/01 PURCHASE 800-257-2582 MA D...,None,"=IF(H$6=$D11,$B11,0)","=IF(I$6=$D11,$B11,0)","=IF(J$6=$D11,$B11,0)","=IF(K$6=$D11,$B11,0)",...,"=IF(AA$6=$D11,$B11,0)","=IF(AB$6=$D11,$B11,0)","=IF(AC$6=$D11,$B11,0)",None,None,None,None,None,None,None
5,2023-07-03 00:00:00,-11.98,???,None,UBER TRIP 07/02 PURCHASE HELP.UBER.COM CA DEBI...,None,"=IF(H$6=$D12,$B12,0)","=IF(I$6=$D12,$B12,0)","=IF(J$6=$D12,$B12,0)","=IF(K$6=$D12,$B12,0)",...,"=IF(AA$6=$D12,$B12,0)","=IF(AB$6=$D12,$B12,0)","=IF(AC$6=$D12,$B12,0)",None,None,None,None,None,None,None
6,2023-07-03 00:00:00,-14.36,???,None,UBER EATS 07/02 PURCHASE HELP.UBER.COM CA DEBI...,None,"=IF(H$6=$D13,$B13,0)","=IF(I$6=$D13,$B13,0)","=IF(J$6=$D13,$B13,0)","=IF(K$6=$D13,$B13,0)",...,"=IF(AA$6=$D13,$B13,0)","=IF(AB$6=$D13,$B13,0)","=IF(AC$6=$D13,$B13,0)",None,None,None,None,None,None,None
7,2023-07-03 00:00:00,-279.19,sum food,None,COSTCO WHSE #0333 07/02 PURCHASE EVERETT MA DE...,None,"=IF(H$6=$D14,$B14,0)","=IF(I$6=$D14,$B14,0)","=IF(J$6=$D14,$B14,0)","=IF(K$6=$D14,$B14,0)",...,"=IF(AA$6=$D14,$B14,0)","=IF(AB$6=$D14,$B14,0)","=IF(AC$6=$D14,$B14,0)",None,None,None,None,None,None,None
8,2023-07-05 00:00:00,-149.74,house,None,THE HOME DEPOT #2688 07/02 PURCHASE EVERETT MA...,None,"=IF(H$6=$D15,$B15,0)","=IF(I$6=$D15,$B15,0)","=IF(J$6=$D15,$B15,0)","=IF(K$6=$D15,$B15,0)",...,"=IF(AA$6=$D15,$B15,0)","=IF(AB$6=$D15,$B15,0)","=IF(AC$6=$D15,$B15,0)",None,None,None,None,None,None,None
9,2023-07-05 00:00:00,-53.23,sum food,None,STAR MARKET 0602 07/03 PURCHASE BOSTON MA DEBI...,None,"=IF(H$6=$D16,$B16,0)","=IF(I$6=$D16,$B16,0)","=IF(J$6=$D16,$B16,0)","=IF(K$6=$D16,$B16,0)",...,"=IF(AA$6=$D16,$B16,0)","=IF(AB$6=$D16,$B16,0)","=IF(AC$6=$D16,$B16,0)",None,None,None,None,None,None,None
10,2023-07-05 00:00:00,-1918.93,electricity,None,Eversource Bill Payment,None,"=IF(H$6=$D17,$B17,0)","=IF(I$6=$D17,$B17,0)","=IF(J$6=$D17,$B17,0)","=IF(K$6=$D17,$B17,0)",...,"=IF(AA$6=$D17,$B17,0)","=IF(AB$6=$D17,$B17,0)","=IF(AC$6=$D17,$B17,0)",None,None,None,None,None,None,None
